In [20]:
import pandas as pd
import os

# Load the dataset
script_dir = os.path.abspath('')
relative_path = 'fr-en-calendrier-scolaire.csv'
csv_path = os.path.join(script_dir, relative_path)
data = pd.read_csv(csv_path, delimiter=";")

In [28]:
# Il y a des vacances où Zone A est répétée plusieurs fois pour la même année et la même description.
data['start_year'] = pd.to_datetime(data['start_date'], utc=True).dt.year
filtered_data = data.drop_duplicates(subset=['description', 'zones', 'start_year'])
filtered_data = filtered_data.drop(columns=['start_year'])

# Filtrer les lignes où 'population' est 'Élèves' ou vide
filtered_data = filtered_data[filtered_data['population'].isin(['Élèves', '-'])]
filtered_data = filtered_data[filtered_data['start_date'].notnull()]
filtered_data = filtered_data[filtered_data['end_date'].notnull()]
filtered_data = filtered_data[filtered_data['zones'].isin(['Zone A', 'Zone B', 'Zone C'])]
filtered_data.to_csv('final_vacation_ratio_filtered.csv', index=False)

# Créer un dictionnaire pour les jours de vacances par année et mois
vacation_days = {}

# Parcourir les données filtrées
for _, row in filtered_data.iterrows():
    start_date = pd.to_datetime(row['start_date'])
    end_date = pd.to_datetime(row['end_date'])
    current_date = start_date
    while current_date <= end_date:
        year = current_date.year
        month = current_date.month
        if year not in vacation_days:
            vacation_days[year] = {}
        if month not in vacation_days[year]:
            vacation_days[year][month] = 0
        vacation_days[year][month] += 1
        current_date += pd.DateOffset(days=1)

# Créer un DataFrame à partir du dictionnaire de jours de vacances
result_df = pd.DataFrame([(year, month, count) for year, months in vacation_days.items()
                          for month, count in months.items()],
                         columns=['annee', 'mois', 'jours_vacances'])

# Calculer le nombre total de jours dans chaque mois
result_df['total_jours'] = result_df.apply(lambda row: pd.Timestamp(year=row['annee'], month=row['mois'], day=1).days_in_month, axis=1)

# Calculer le ratio de jours de vacances par rapport au nombre total de jours
result_df['ratio_vacances'] = result_df['jours_vacances'] / result_df['total_jours']

# Afficher le résultat
print(result_df)

    annee  mois  jours_vacances  total_jours  ratio_vacances
0    2017    10              33           31        1.064516
1    2017    11              18           30        0.600000
2    2017    12              27           31        0.870968
3    2018     1              24           31        0.774194
4    2018     2              34           28        1.214286
..    ...   ...             ...          ...             ...
84   2026     2              40           28        1.428571
85   2026     3              11           31        0.354839
86   2026     4              47           30        1.566667
87   2026     5              10           31        0.322581
88   2026     7               3           31        0.096774

[89 rows x 5 columns]


In [29]:
# Pour enregistrer le résultat dans un nouveau fichier CSV
result_df.to_csv('vacation_month_ratio.csv', index=False)